# LIDC-IDRI SimpleITK

For working with dicom files we now use SimpleITK library which is suited for working with medical formats.

In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

In [ ]:
path = "../LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192"

Creation of a Image Series Reader that will allow us to work with a full volume.

In [3]:
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(path)
reader.SetFileNames(dicom_names)
image = reader.Execute()

In [4]:
original_size = image.GetSize()
original_spacing = image.GetSpacing()

original_size, original_spacing

((512, 512, 133), (0.703125, 0.703125, 2.5))

From a specific slice of the volume and rescaling to 512 px height, the width is calculated to maintain proportions

In [5]:
new_spacing = [
    (original_size[0] - 1) * original_spacing[0] / (512 - 1),
] * 2 + [original_spacing[2]]
new_size = [512, int((original_size[1] - 1) * original_spacing[1] / new_spacing[1]), 1]

new_size, new_spacing

([512, 511, 1], [0.703125, 0.703125, 2.5])

Resample the image using a linear transformation and save it to a PNG file

In [6]:
image = sitk.Resample(
    image1=image,
    size=new_size,  # type: ignore
    transform=sitk.Transform(),
    interpolator=sitk.sitkLinear,
    outputOrigin=image.GetOrigin(),
    outputSpacing=new_spacing,
    outputDirection=image.GetDirection(),
    defaultPixelValue=0,
    outputPixelType=image.GetPixelID(),
)

if image.GetNumberOfComponentsPerPixel() == 1:
    image = sitk.RescaleIntensity(image, 0, 255)
    image = sitk.Cast(image, sitk.sitkUInt8)
    sitk.WriteImage(image, "image.png")